In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [5]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = json.loads(open(data_path, "r").read())
x1_vals = np.array(data['x1'])
x2_vals = np.array(data['x2'])
y_vals = np.array(data['y'])

createDummy(data_path, dummy_data_path)
dummy_data = json.loads(open(dummy_data_path, "r").read())
dummy_x1_vals = np.array(dummy_data['x1'])
dummy_x2_vals = np.array(dummy_data['x2'])
dummy_y_vals = np.array(dummy_data['y'])



def stacked_x(*args):
    result = np.column_stack((*args, np.ones_like(args[0])))
    return result

lt20_y_vals = y_vals[y_vals<20.0]
lt20_dummy_y_vals = dummy_y_vals[dummy_y_vals<20.0]

x_one = stacked_x(x1_vals, x2_vals)
dummy_x_one = stacked_x(dummy_x1_vals, dummy_x2_vals)
lt20_x_one = stacked_x(x1_vals[y_vals<20.0], x2_vals[y_vals<20.0])
lt20_dummy_x_one = stacked_x(dummy_x1_vals[dummy_y_vals<20.0], dummy_x2_vals[dummy_y_vals<20.0])

w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(x_one.transpose(), x_one)), x_one.transpose()), y_vals)
dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(dummy_x_one.transpose(), dummy_x_one)), dummy_x_one.transpose()), dummy_y_vals)
lt20_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(lt20_x_one.transpose(), lt20_x_one)), lt20_x_one.transpose()), lt20_y_vals)
lt20_dummy_w_vals = np.matmul(np.matmul(np.linalg.inv(np.matmul(lt20_dummy_x_one.transpose(), lt20_dummy_x_one)), lt20_dummy_x_one.transpose()), lt20_dummy_y_vals)


print("reg fit: ", w_vals)
print("lt 20 reg fit: ", lt20_w_vals)

lt20_w_tensor = torch.tensor(lt20_w_vals).reshape(1,-1,1)
lt20_dummy_w_tensor = torch.tensor(lt20_dummy_w_vals).reshape(1,-1,1)



reg fit:  [1.98835287 3.14238058 0.06413647]
lt 20 reg fit:  [ 2.12311694  3.43333008 -1.03203902]


In [9]:
# Verifier/ data consumer side:
# Want to calculate regression of y over 
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        self.w = nn.Parameter(data = lt20_dummy_w_tensor, requires_grad = False)
        
    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        filter = (Y<20)
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0])), dim=2)
        fil_X_one = torch.where(filter, X_one, torch.tensor([0.0]*X_one.size()[2]))
        fil_X_T = torch.transpose(fil_X_one, 1, 2)
        fil_Y = torch.where(filter, Y, 0.0)

        return (
            torch.sum(torch.abs(fil_X_T @ fil_X_one @ self.w - fil_X_T @ fil_Y)) <= 0.01 * torch.sum(torch.abs(fil_X_T @ fil_Y)),
            self.w
        )

verifier_define_calculation(dummy_data_path, ['x1','x2','y'],sel_dummy_data_path,verifier_model, verifier_model_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_13580/713816729.py:13: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  fil_X_one = torch.where(filter, X_one, torch.tensor([0.0]*X_one.size()[2]))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Where, node name: /Where): Y has inconsistent type tensor(float) (Triggered internally at /Users/runner/work/p

In [10]:
# prover calculates settings, send to verifier

theory_output = lt20_w_tensor
print("Theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        self.w = nn.Parameter(data = lt20_w_tensor, requires_grad = False)

    def forward(self, *args):
        # infer Y from the last parameter
        Y = args[-1]
        filter = (Y<20)
        X_one = torch.cat((*args[:-1], torch.ones_like(args[0])), dim=2)
        fil_X_one = torch.where(filter, X_one, torch.tensor([0.0]*X_one.size()[2]))
        fil_X_T = torch.transpose(fil_X_one, 1, 2)
        fil_Y = torch.where(filter, Y, 0.0)

        return (
            torch.sum(torch.abs(fil_X_T @ fil_X_one @ self.w - fil_X_T @ fil_Y)) <= 0.01 * torch.sum(torch.abs(fil_X_T @ fil_Y)),
            self.w
        )
    
prover_gen_settings(data_path,['x1','x2','y'], sel_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

Theory output:  tensor([[[ 2.1231],
         [ 3.4333],
         [-1.0320]]], dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [0]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":0,"param_scale":0,"scale_rebase_multiplier":10,"lookup_range":[-5526,4698],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":11808,"total_assignments":1694,"total_const_size":115,"model_instance_shapes":[[1],[1,3,1]],"model_output_scales":[0,0],"model_input_scales":[0,0,0],"module_sizes":{"kzg":[],"poseidon":[11808,[3]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_13580/2840331579.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  fil_X_one = torch.where(filter, X_one, torch.tensor([0.0]*X_one.size()[2]))


In [11]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0


spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.9537100791931152 seconds
Theory output:  tensor([[[ 2.1231],
         [ 3.4333],
         [-1.0320]]], dtype=torch.float64)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 2.0
witness result 2 : 3.0
witness result 3 : -1.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [10902020042510041094, 17381486299841078119, 5900175412809962030, 2475245527108272378]]], 'proof': '012fc9e71e694abb4c641d8af1f4b353230eb0c4c2a064f02f37338d71f0a7601579f60c7627835113c6695e07c51f7903ce9a8fa625df7233c3b90f4d455ad31762e2416c6c050d906f57b988542d654fece250289323b3533c533e5949c9212a70041c92040058c2c4107c49cf55a57231de7c01f18f2ee6352de4b35e580324499d06e59fbe4c6a9ac38436950ff15a21a103c03f4731e1719ca3410af735238ec916176cdf146ab058173fcb09ec7d855b

In [12]:
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  3
prf instances:  [[[6145674602038562713, 11796601694033167407, 3132644448460071153, 1431119980703310933], [17187590983289934876, 11857991285122296962, 971807162298867662, 379283799527326290], [3957842973089931008, 9845595232537184463, 786695466761881781, 2995319695946854765], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [6425625360762666998, 7924344314350639699, 14762033076929465436, 2023505479389396574], [415066004289224689, 11886516471525959549, 3696305541684646538, 3035258219084094862], [10902020042510041094, 17381486299841078119, 5900175412809962030, 2475245527108272378]]]
proof boolean:  1.0
proof result 1 : 2.0
proof result 2 : 3.0
proof result 3 : -1.0
verified
